In [ ]:
!pip install deepl decouple contractions spacy tqdm deep-translator

In [11]:
import json
import os
import re
import string

import contractions
import pandas as pd
import spacy
import text2emotion as te

from deep_translator import GoogleTranslator as Translator
from nltk.tokenize import word_tokenize

In [12]:
pd.set_option('display.max_columns', None)

# Get datasets into single dictionary

In [23]:
path="./data-extraction/"
df_dict = {}
for root, dirs, files in os.walk(path):
    for file_name in files:
        if file_name.endswith('.jl') or file_name.endswith('.jsonl'):
            file_path = os.path.join(root, file_name)
            name = os.path.splitext(file_name)[0]
            df_dict[name] = pd.read_json(path_or_buf=file_path, lines=True)

In [24]:
type(df_dict)

dict

In [25]:
for df in df_dict:
    df_dict[df] = df_dict[df].drop("url", axis=1).drop("_type", axis=1).drop("cashtags", axis=1) #.head() # to sample the data down for testing this monster
    keys = df_dict[df].keys()
    #print(f"{df} {keys}")

# Data Cleaning

In [26]:
def replace_strings(column: str, to_replace: str, value: str, regex: bool) -> pd.DataFrame:
    return df[column].replace(to_replace=to_replace, value=value, regex=regex)

In [27]:
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\n', ' ', text)
    return text.lower()

In [28]:
def split_pascal_case(word):
    return re.sub(r'(?<=\w)([A-Z])', r' \1', word)

In [29]:
translator = Translator(target="en")
def get_translations(x):
    if x["lang"] == "en":
        return x["cleanedContent"]
    else:
        translation = translator.translate(x["cleanedContent"])
        return translation

In [30]:
for frame in df_dict:
    print(f"Current dataframe: {frame}")

    df_dict[frame].fillna(value='', inplace=True)
    
    # Convert column to string type
    df_dict[frame]['cleanedContent'] = df_dict[frame]['rawContent'].astype(str)
    
    # Clean and translate the data
    df_dict[frame]["cleanedContent"] = df_dict[frame]['cleanedContent'].str.replace(r'\bmpox\b', 'monkeypox', regex=True)
    df_dict[frame]["cleanedContent"] = df_dict[frame]['cleanedContent'].apply(split_pascal_case)
    df_dict[frame]["cleanedContent"] = df_dict[frame]['cleanedContent'].apply(remove_special_characters)
    # df_dict[frame]["translatedContent"] = df_dict[frame].apply(get_translations, axis=1)
    
    # Add tokenized column
    df_dict[frame]["tokens"] = df_dict[frame]["cleanedContent"].apply(word_tokenize)
    
    # Save to files
    df_dict[frame].to_json(f"./data-cleaned/processed_{frame}.json", orient='records', lines=True)

Current dataframe: twitter-pfizer
Current dataframe: tw_hshtag_covid19
Current dataframe: tw_hshtag_monkeypox
Current dataframe: tw_hshtag_flu


In [ ]:
# do the pfizer dataset without translation and just do a emotion check, what pfizer sentiment is